# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import random

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kovalam,8.3667,76.9967,27.05,30.15,75,4.12,IN,1688617768
1,1,waitangi,-43.9535,-176.5597,9.56,4.55,82,16.21,NZ,1688617770
2,2,benguela,-12.5763,13.4055,18.25,18.43,1,0.62,AO,1688617771
3,3,ushuaia,-54.8000,-68.3000,1.81,-2.14,40,4.12,AR,1688617772
4,4,isafjordur,66.0755,-23.1240,6.03,3.44,100,3.46,IS,1688617773


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
baseurl = "https://api.geoapify.com/v1/forecast/weather"
params = {"apiKey": geoapify_key}


# Configure the map plot
city_data_df["colour"] = ""

for index, row in city_data_df.iterrows():
    city_data_df.at[index,"colour"] = f"#{random.randint(0,0xFFFFFF):06x}"

# Display the map
city_data_df.hvplot.points(x="Lng", y="Lat", 
                           size="Humidity", geo=True, tiles="CartoLight",color="colour",alpha = 0.5)

c:\Users\gello\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
c:\Users\gello\anaconda3\envs\dev\lib\site-packages\holoviews\util\transform.py:674: RuntimeWarning: invalid value encountered in sqrt
  data = fn(*args, **kwargs)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (colour,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
#Criteria: max temp: 21-27, wind speed: < 4.5, cloudiness == 0.
filtered_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
# Drop any rows with null values
].dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,colour
8,8,bethel,41.3712,-73.4140,23.83,22.96,0,0.00,US,1688617580,#fca826
11,11,hamilton,39.1834,-84.5333,25.31,24.16,0,2.06,US,1688617493,#c8c40f
116,116,jamestown,42.0970,-79.2353,21.80,21.54,0,0.45,US,1688617812,#045688
125,125,riverton,40.5219,-111.9391,26.13,22.07,0,2.57,US,1688617707,#eddb23
132,132,zarghun shahr,32.8473,68.4457,25.63,24.93,0,2.84,AF,1688617962,#3bb8a3


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,bethel,US,41.3712,-73.4140,22.96,
11,hamilton,US,39.1834,-84.5333,24.16,
116,jamestown,US,42.0970,-79.2353,21.54,
125,riverton,US,40.5219,-111.9391,22.07,
132,zarghun shahr,AF,32.8473,68.4457,24.93,
188,demopolis,US,32.5176,-87.8364,26.13,
240,kamianske,UA,48.5113,34.6021,26.63,
267,show low,US,34.2542,-110.0298,20.24,
296,al bawiti,EG,28.3492,28.8659,26.05,
298,nadym,RU,65.5333,72.5167,23.50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":4,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"circle:{lat},{lon},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["filter"] = f"circle:{lon},{lat},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
hamilton - nearest hotel: North Vista Manor
jamestown - nearest hotel: DoubleTree Jamestown
riverton - nearest hotel: Ramada by Wyndham Draper
zarghun shahr - nearest hotel: No hotel found
demopolis - nearest hotel: No hotel found
kamianske - nearest hotel: Шиллер
show low - nearest hotel: Holiday Inn Express Showlow
al bawiti - nearest hotel: Old Oasis Hotel
nadym - nearest hotel: Янтарная
verde village - nearest hotel: Valley Verde Inn
mayfield - nearest hotel: Wingfield Inn
little falls - nearest hotel: Holiday Inn Totowa Wayne
hampton bays - nearest hotel: Canoe Place
newton - nearest hotel: Four Points by Sheraton Boston Newton
enid - nearest hotel: La Quinta Inn & Suites by Wyndham Enid
laurel - nearest hotel: Quality Inn & Suites
west point - nearest hotel: No hotel found
theodore - nearest hotel: InTown Suites Extended Stay Mobile AL - Inn Road
roma - nearest hotel: Wingate by Wyndham Rome
hozat - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
8,bethel,US,41.3712,-73.4140,22.96,Hampton Inn Danbury
11,hamilton,US,39.1834,-84.5333,24.16,North Vista Manor
116,jamestown,US,42.0970,-79.2353,21.54,DoubleTree Jamestown
125,riverton,US,40.5219,-111.9391,22.07,Ramada by Wyndham Draper
132,zarghun shahr,AF,32.8473,68.4457,24.93,No hotel found
188,demopolis,US,32.5176,-87.8364,26.13,No hotel found
240,kamianske,UA,48.5113,34.6021,26.63,Шиллер
267,show low,US,34.2542,-110.0298,20.24,Holiday Inn Express Showlow
296,al bawiti,EG,28.3492,28.8659,26.05,Old Oasis Hotel
298,nadym,RU,65.5333,72.5167,23.50,Янтарная


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)